In [54]:
import collections


first_file_path = "../Fajlovi/1000G_test.vcf"
second_file_path = "../Fajlovi/1000G_test_2.vcf"

merged_vcf_file = open("merged.vcf","w+")


#možda da napravim jednu veliku klasu VCF_file u kojoj ću imati metode za sortiranje dictionaries-a filters i formats

class File_format: 
    def __init__(self, line):
        self.line = line
        self.vcf_version = ""
        self.extract_file_format_version()
        
    def __eq__(self, other):
        return self.line == other.line
    
    def extract_file_format_version(self):
        try:
            version = self.line.split("=VCFv",1)[1]
            self.vcf_version = version
        except:
            pass
    

class Filter_header:
    def __init__(self, line):
        self.line = line
        self.id = ""
        self.description = ""
        self.extract_id()
        self.extract_description()
        
    def extract_id(self):
        try:
            id = (self.line.split("ID="))[1].split(",")[0]
            self.id = id
        except:
            pass
    
    def extract_description(self):
        try:
            description = (self.line.split('Description="'))[1].split('"')[0]
            self.description = description
        except:
            pass

# ##FORMAT=<ID=GT,Number=1,Type=String,Description="Genotype">
class Format_header:
    def __init__(self, line):
        self.line = line
        self.id = ""
        self.number = ""
        self.type = ""
        self.description = ""
        self.extract_id()
        self.extract_number()
        self.extract_type()
        self.extract_description()
        
    def extract_id(self):
        try:
            self.id = (self.line.split("ID="))[1].split(",")[0]
        except:
            pass
    
    def extract_number(self):
        try:
            self.number = (self.line.split("Number="))[1].split(",")[0]            
        except:
            pass
        
    def extract_type(self):
        try:
            self.type = (self.line.split("Type="))[1].split(",")[0]            
        except:
            pass
    
    def extract_description(self):
        try:
            self.description = (self.line.split('Description="'))[1].split('"')[0]
        except:
            pass
    
class Filter_lifted_variant:
    def __init__(self, line):
        self.line = line
        #da li nam trebaju sva ona pojedinačna polja
    
    
# ##INFO=<ID=GentrainScore,Number=.,Type=Float,Description="Gentrain Score">
class Info_header:
    def __init__(self, line):
        self.line = line
        self.id = ""
        self.number = ""
        self.type = ""
        self.description = ""
        self.extract_id()
        self.extract_number()
        self.extract_type()
        self.extract_description()
        
    def extract_id(self):
        try:
            self.id = (self.line.split("ID="))[1].split(",")[0]
        except:
            pass
    
    def extract_number(self):
        try:
            self.number = (self.line.split("Number="))[1].split(",")[0]            
        except:
            pass
        
    def extract_type(self):
        try:
            self.type = (self.line.split("Type="))[1].split(",")[0]            
        except:
            pass
    
    def extract_description(self):
        try:
            self.description = (self.line.split('Description="'))[1].split('"')[0]
        except:
            pass
        
# ##contig=<ID=chr13,length=114364328,assembly=20>
class Contig_header:
    def __init__(self, line):
        self.line = line
        self.id = ""
        self.length = ""
        self.assembly = ""
        self.extract_id()
        self.extract_length()
        self.extract_assembly()
        
    def extract_id(self):
        try:
            self.id = (self.line.split("ID="))[1].split(",")[0]
        except:
            pass
    
    def extract_length(self):
        try:
            self.number = (self.line.split("length="))[1].split(",")[0]            
        except:
            pass
        
    def extract_assembly(self):
        try:
            self.type = (self.line.split("assembly="))[1].split(">")[0]            
        except:
            pass

    #do I need to close merged_file ???
    #get rid of filters, GarbageCollector or just filters = {}

class Input_file:
    current_tag = ""
    
    def __init__(self, path):
        self.path = path
        self.has_current_tag = False
        self.header_read = False
    
    def set_file(self, file):
        self.file = file
    
    def set_next_tag(self, next_tag):
        self.next_tag = next_tag
        
    def set_value_has_current_tag(self):
        self.has_current_tag = (self.current_tag == self.next_tag)
        

#pošto se filteri trebaju sortirati trebam kupiti filtere u jednu listu 
#da li ovdje koristiti listu ili nešto drugo, uzećemo dictionary
#kod filtera sortiranje ide po ID 
file_formats = {}
filters = {}
formats = {}
infos = {}
contigs = {}
filter_lifted_variants = []

#WILDIDEA
#da li napraviti da je izlazni fajl kopija većeg fajla i onda dodavati dodatne linije
   
#ovo moramo napraiti da bude samo jedna funkcija, da može da radi i prvi i drugi fajl, a kasnije i proizvoljan niz fajlova
def read_specific_tag_in_files(tag, list_of_files):
    
    for item in list_of_files:
        if(item.has_current_tag):
            previous_position_of_file = item.file.tell()
            line_of_file = item.file.readline()
        
            while(line_of_file.startswith("##" + tag)):
                create_object_add_to_dict(tag, line_of_file)
                previous_position_of_file = item.file.tell()
                line_of_file = item.file.readline()
            
            #rewind file 
            item.file.seek(previous_position_of_file)
    
    write_tag_in_file(tag)        
    
def write_tag_in_file(tag):
    if(tag == "fileformat"):
        write_vcf_version()
    elif(tag == "FILTER"):
        for key, value in sorted(filters.items()):
            merged_vcf_file.write(value.line)
    elif(tag == "FORMAT"):
        for key, value in sorted(formats.items()):
            merged_vcf_file.write(value.line)
    elif(tag == "FilterLiftedVariants"):
        for value in filter_lifted_variants:
            merged_vcf_file.write(value.line) 
    elif(tag == "INFO"):
        for key, value in sorted(infos.items()):
            merged_vcf_file.write(value.line)
    elif(tag == "contig"):
        for key, value in sorted(contigs.items()):
            merged_vcf_file.write(value.line)
            
def write_vcf_version():
    line = file_formats[(sorted(file_formats, reverse = True))[0]].line 
    merged_vcf_file.write(line)

def create_object_add_to_dict(tag, line):
    if(tag == "fileformat"):
        file_format = File_format(line)
        file_formats[file_format.vcf_version] = file_format
    elif(tag == "FILTER"):
        filter_header = Filter_header(line)
        if(filter_header not in filters.values()):
            filters[filter_header.id] = filter_header
    elif(tag == "FORMAT"):
        format_header = Format_header(line)
        if(format_header not in formats.values()):
            formats[format_header.id] = format_header  
    elif(tag == "FilterLiftedVariants"):
        filter_lifted_variant = Filter_lifted_variant(line)
        filter_lifted_variants.append(filter_lifted_variant) 
    elif(tag == "INFO"):
        info_header = Info_header(line)
        if(info_header not in infos.values()):
            infos[info_header.id] = info_header
    elif(tag == "contig"):
        contig_header = Contig_header(line)
        if(contig_header not in contigs.values()):
            contigs[contig_header.id] = contig_header 

def get_next_tag(list_of_files):
    
    tags = []
    
    for item in list_of_files:
        previous_position_of_file = item.file.tell()
        
        next_line = item.file.readline()
        if(next_line.split("=")[0].startswith("##")):
            item.set_next_tag(next_line.split("=")[0])
            tags.append(item.next_tag)
        else:
            item.set_next_tag("")
        item.file.seek(previous_position_of_file)
        
    Input_file.current_tag = (list(sorted(tags)))[0] if len(tags) > 0 else ""
    
    #set files which has current tag
    for item in list_of_files:
        item.set_value_has_current_tag()
        
#koliko ima različitih tagova u header-u VCF fajlova     
    
    
first_file = Input_file(first_file_path)
second_file = Input_file(second_file_path)
list_of_files = [first_file, second_file]

not_end_of_header = True
# context manager
with open(list_of_files[0].path) as list_of_files[0].file:
    with open(list_of_files[1].path) as list_of_files[1].file:
        
        while not_end_of_header:
            get_next_tag(list_of_files)
            if Input_file.current_tag.startswith("##"):
                read_specific_tag_in_files(Input_file.current_tag[2:], list_of_files)
            else:
                not_end_of_header = False
        
    